In [2]:
import sys
# add a path to sys.path
sys.path.append('/home/wenxinjiang/tf-maskformer/')
sys.path.append('/home/wenxinjiang/tf-maskformer/models/')
sys.path.append('/home/wenxinjiang/tf-maskformer/models/official/')

In [17]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2
from official.projects.maskformer.dataloaders import panoptic_input
from official.projects.detr.dataloaders import detr_input
import os
import tensorflow as tf
from official.core import train_utils

from official.projects.detr.tasks import detection


from official.core import task_factory

In [4]:
COCO_INPUT_PATH_BASE = 'gs://cam2-datasets/coco'
COCO_ANNOTATION_PATH_BASE = 'gs://cam2-datasets/annotations'
COCO_TRAIN_EXAMPLES = 118287
COCO_VAL_EXAMPLES = 5000


In [5]:
tfrecord_path = COCO_INPUT_PATH_BASE  # specify the filepath to tfrecord
# get list of tfrecord files
file_paths = tf.io.gfile.glob(tfrecord_path + "/*.tfrecord")
decoder = panoptic_input.TfExampleDecoder()
image_count = 0

In [6]:
def test_yolo_input_task():
  # with tf.device('/CPU:0'):
  experiment = "detr_coco_tfrecord"
  config_path = ["/home/wenxinjiang/tf-maskformer/models/official/projects/detr/configs/detr_tpu_v3_640.yaml"]
  # config_path = ["yolo/configs/experiments/yolov4-csp/tpu/640.yaml"]
  # config_path = ["yolo/configs/experiments/yolov4-csp-anchor-free/tpu/640-cstm2.yaml"]

  config = train_utils.ParseConfigOptions(
      experiment=experiment, config_file=config_path)
  params = train_utils.parse_configuration(config)
  config = params.task

  # anchor gen testing
  # config.model.boxes = None

  task = task_factory.get_task(params.task)

  config.train_data.global_batch_size = 1
  config.validation_data.global_batch_size = 1

  config.train_data.dtype = 'float32'
  config.validation_data.dtype = 'float32'

  config.validation_data.shuffle_buffer_size = 1
  config.train_data.shuffle_buffer_size = 1


  config.train_data.input_path = COCO_INPUT_PATH_BASE+'/train*'
  config.validation_data.input_path = COCO_INPUT_PATH_BASE+'/val*'

  with tf.device('/CPU:0'):
    train_data = task.build_inputs(config.train_data)
    test_data = task.build_inputs(config.validation_data)
  return train_data, test_data, config

In [7]:
dataset, dsp, config = test_yolo_input_task()

2023-08-26 16:55:04.579863: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/envs/maskformer/lib/python3.9/site-packages/cv2/../../lib64:
2023-08-26 16:55:04.579897: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-26 16:55:04.579914: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (compute-intensive-1): /proc/driver/nvidia/version does not exist
2023-08-26 16:55:04.580207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild Te

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [43]:
def coco_to_tf_format(boxes):
    tf_boxes = []
    for box in boxes:
        x1, y1, width, height = box
        tf_boxes.append([y1, x1, y1 + height, x1 + width])
    return tf.convert_to_tensor(tf_boxes, dtype=tf.float32)


In [56]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_bounding_boxes_on_image(ax, image, boxes):
    """
    Draws bounding boxes on image.
    Args:
    - ax: A matplotlib axis to draw upon.
    - image: A [H, W, 3] RGB image.
    - boxes: A list of bounding boxes in the format (cx, cy, w, h).
    """
    image = (image * 255).astype(np.uint8)
    ax.imshow(image)
    
    for box in boxes:
        # Convert (cx, cy, w, h) to (x_min, y_min, x_max, y_max)
        cy, cx, w, h = box
        x_min, y_min = int((cx - w / 2) * image.shape[1]), int((cy - h / 2) * image.shape[0])
        x_max, y_max = int((cx + w / 2) * image.shape[1]), int((cy + h / 2) * image.shape[0])

        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                 linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

def test_yolo_pipeline(is_training=True, num=5):
    dataset, dsp, config = test_yolo_input_task()  # assuming this function provides required data

    data = dataset if is_training else dsp
    data = data.take(num)
    for l, (i, j) in enumerate(data):
        for shind in range(1):  # loop runs only once
            fig, axe = plt.subplots(1, 2)

            image = i[shind].numpy()
            # Denormalize the image from [0,1] to [0,255]
            image = (image * 255).astype(np.uint8)
            boxes = j["boxes"][shind].numpy()

            draw_bounding_boxes_on_image(axe[0], image, boxes)

            fig.set_size_inches(18.5, 6.5, forward=True)
            plt.tight_layout()
            plt.savefig(f"test{l}.png")
            plt.show()
            plt.close()


In [57]:
test_yolo_pipeline()

TypeError: 'Axes' object is not subscriptable